In [1]:
import os
import time

# Self-code imports
import polytopes.segmentation_algorithms as algos
from polytopes.model.note import Note
from polytopes.model.chord import Chord
import polytopes.polytopical_costs as pc
import polytopes.data_manipulation as dm
import polytopes.pattern_manip as pm
import polytopes.pattern_factory as pf
import polytopes.segmentation_helper as sh

#Generic imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy

In [2]:
# Hyperparams
max_size = 47
min_size = 8
positive_segment_size_penalty = 0
negative_segment_size_penalty = 0#.125
target_size = 32
positive_penalty = 0#2.25
negative_penalty = 0#3

# On a song

In [3]:
flow_file_path = "C:\\Users\\amarmore\\Desktop\\Annotations MIREX Quaero\\annotations_as_flow\\"
song = "001"
bag_of_chords = []
file = open(flow_file_path + song + ".flow", "r") 
for chord in file.readlines():
    bag_of_chords.append(chord.split("\n")[0])

In [4]:
frontiers, costs = algos.dynamic_minimization_cohen_marmoret(bag_of_chords)

print("My frontiers: {}".format(frontiers))
print("Costs: {}".format(costs))

guichaoua_estimates = open(flow_file_path + "Guichaoua Estimates\\Experiments\\" + song + ".frontiers_estimated", "r")
tab = guichaoua_estimates.read().split(" ")
guichaoua_frontiers = [int(i) - 1 for i in tab if i != "\n"]
guichaoua_frontiers.append(frontiers[-1])
print("Guichaoua's frontiers: {}".format(guichaoua_frontiers))
print("Equality ? {}\n".format(guichaoua_frontiers == frontiers))

c:\users\amarmore\desktop\projects\phd main projects\on git\code\polytopes and relation\polytopes\segmentation_algorithms.py:239: UserWarning: No Polytope is available for this size of segment. Trying sequential cost instead.
  warnings.warn("No Polytope is available for this size of segment. Trying sequential cost instead.")


My frontiers: [0, 24, 52, 84, 116, 148, 180, 192, 224, 256, 284, 315, 336, 368, 396, 428, 462]
Costs: 425
Guichaoua's frontiers: [0, 20, 52, 84, 112, 144, 176, 192, 220, 252, 284, 292, 332, 364, 396, 428, 462]
Equality ? False



In [6]:
the_sum = 0
for start, end in zip(frontiers[:-1], frontiers[1:]):
    segment = bag_of_chords[start:end]
    #print(segment)
    size = len(segment)
    print(segment)
    bag_of_patterns = sh.compute_patterns_with_antecedents_for_size(size)
    the_min = math.inf
    best_pattern = None
    for a_pattern in bag_of_patterns:
        this_polytope_cost = math.inf
        for i in range(len(a_pattern[0])):
            this_ppp_cost = pc.cohen_marmoret_cost(segment, a_pattern[0], a_pattern[3])
            if this_ppp_cost < this_polytope_cost:
                this_polytope_cost = this_ppp_cost
        cost = this_polytope_cost
        cost += pc.irregularities_penalty_guichaoua(adding_code = a_pattern[1], deleting_code = a_pattern[2], positive_penalty = positive_penalty, negative_penalty = negative_penalty)
        if cost < the_min:
            best_pattern = a_pattern
            the_min = cost
    the_min += sh.penalty_cost_guichaoua(size, target_size = 32, positive_segment_size_penalty = positive_segment_size_penalty, negative_segment_size_penalty = negative_segment_size_penalty)
    if bag_of_patterns == []:
        print("Sequential cost")
        break
    print("Sequence: {}".format(pf.apply_chords_on_pattern(best_pattern[0],segment)))
    print("Cost: {} at size {}".format(the_min, end-start))
    the_sum += the_min
print("Ze sum:{}".format(the_sum))

['Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'E', 'E', 'E', 'E', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'Eb', 'Eb', 'Eb', 'Eb']
Sequence: [[[[['Abm', 'Abm'], ['Abm', 'Abm']], [['F#', 'F#'], ['F#', 'F#']]], [[['E', 'E'], ['E', 'E']], [['F#', 'F#'], ['F#', 'F#']]]], [[[['F#', 'F#'], ['F#', 'F#']], [['Eb', 'Eb'], ['Eb', 'Eb']]]]]
Cost: 16 at size 24
['Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'B', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb']
Sequence: [[[[['Abm', 'Abm'], ['Abm', 'Abm']], [['E', 'E'], ['E', 'E']]], [[['B', 'B'], ['B', 'B']], [['Eb', 'Eb'], ['Eb', 'Eb']]]], [[[['Abm', 'Abm'], ['Abm', 'Abm']], [['E', 'E'], ['E', 'E']]], [[['F#', 'F#']], [['Eb', 'Eb']]]]]
Cost: 23 at size 28
['Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#', 'Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#']
Sequence: [[[[['Abm',

In [8]:
the_sum = 0
for start, end in zip(guichaoua_frontiers[:-1], guichaoua_frontiers[1:]):
    segment = bag_of_chords[start:end]
    #print(segment)
    size = len(segment)
    print(segment)
    bag_of_patterns = sh.compute_patterns_with_antecedents_for_size(size)
    the_min = math.inf
    best_pattern = None
    for a_pattern in bag_of_patterns:
        this_polytope_cost = math.inf
        for i in range(len(a_pattern[0])):
            this_ppp_cost = pc.cohen_marmoret_cost(segment, a_pattern[0], a_pattern[3])
            if this_ppp_cost < this_polytope_cost:
                this_polytope_cost = this_ppp_cost
        cost = this_polytope_cost
        cost += pc.irregularities_penalty_guichaoua(adding_code = a_pattern[1], deleting_code = a_pattern[2], positive_penalty = positive_penalty, negative_penalty = negative_penalty)
        if cost < the_min:
            best_pattern = a_pattern
            the_min = cost
    the_min += sh.penalty_cost_guichaoua(size, target_size = 32, positive_segment_size_penalty = positive_segment_size_penalty, negative_segment_size_penalty = negative_segment_size_penalty)
    if bag_of_patterns == []:
        print("Sequential cost")
        break
    print("Sequence: {}".format(pf.apply_chords_on_pattern(best_pattern[0],segment)))
    print("Cost: {} at size {}".format(the_min, end-start))
    the_sum += the_min
print("Ze sum:{}".format(the_sum))

['Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'E', 'E', 'E', 'E', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#', 'F#']
Sequence: [[[['Abm', 'Abm'], ['Abm', 'Abm']], [['F#', 'F#'], ['F#', 'F#']]], [[['E', 'E'], ['E', 'E']], [[('F#', 'F#'), ('F#', 'F#')], [('F#', 'F#'), ('F#', 'F#')]]]]
Cost: 7 at size 20
['Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'B', 'B', 'B', 'B', 'Eb', 'Eb', 'Eb', 'Eb', 'Abm', 'Abm', 'Abm', 'Abm', 'E', 'E', 'E', 'E', 'F#', 'F#', 'Eb', 'Eb']
Sequence: [[[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['B', 'B'], [('B', 'B')]]]], [[[['Eb', 'Eb'], ['Eb', 'Eb']], [['Abm', 'Abm'], ['Abm', 'Abm']]], [[['E', 'E'], ['E', 'E']], [['F#', 'F#'], [('Eb', 'Eb')]]]]]
Cost: 39 at size 32
['Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#', 'Abm', 'Abm', 'Abm', 'Abm', 'F#', 'F#', 'F#', 'F#', 'C#', 'C#', 'C#', 'C#', 'E', 'E', 'F#', 'F#']
Sequence: [[[[['Abm', 'A